# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [12]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
"""
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-in-dynamically-loading-webpage/48851166#48851166
"""

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [13]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [14]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 中油宣佈下周油價降8角 九五汽油跌破20元大關

▲國際油價走勢近期波動劇烈，台灣中油宣佈調降下周油價。（圖／記者林敬旻攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情重創全球經濟，衝擊石油市場需求，台灣中油今（29）日宣佈下周汽、柴油每公升零售價分別調降0.8元，為連續五周調降油價，九五無鉛汽油每公升為19.7元，已跌破20元大關。
依據中油目前油價估算，調整後的油價分別為92無鉛汽油每公升18.2元、95無鉛汽油每公升19.7元、98無鉛汽油每公升21.7元、超級柴油每公升15.4元。
台灣中油表示，本週受沙俄等油國削價增產、美國各州COVID-19 病例激增等因素影響，造成國際原油價格下跌。本週調價指標7D3B週均價為每桶24.99美元，較前週28.01美元下跌3.02美元，新臺幣兌美元匯率貶值0.007元。
----------------------------------------------------------------------

[1] 友邦保險2019年淨利年增1.5倍　「低息環境+肺炎衝擊」影響今年首季新業務銷售

▲AIA友邦保險。（圖／路透社）
記者楊絡懸／台北報導
友邦保險2019年度財務業績顯示，全年業績淨利為66.48億美元，按固定匯率計算年增1.5倍。不過，友邦保險認為，市場正面對低息環境帶來一些短期阻力，尤其出現新冠肺炎（COVID-19）疫情，對Q1新業務銷售已造成顯著影響。
友邦公布，2019年度新業務價值41.54億美元，年增6%，不計入香港業務的新業務價值增長16%；年化新保費65.85億美元，年增2%；新業務價值利潤率上升3個百分點至62.9%。至於全年股息，末期息每股派93.3港仙，按年增10%；全年股息總額每股1.266港元，上升11%。
友邦保險集團首席執行官兼總裁黃經輝（Ng Keng Hooi）表示，新業務價值增長6%，基於2019年上半年強勁增長，下半年表現則受到香港社會動蕩的嚴重影響，不計入香港業務，新業務價值增長16%。稅後營運溢利上升9%，期內產生的基本自由盈餘增加13%，而內涵價值權益也增加了12%。
▲友邦保險集團首席執行官兼總

▲台灣連鎖家具龍頭詩肯董事會通過，將配發1.8元現金股利，配發率達八成。圖為詩肯董事長林福勤。（圖／記者李毓康攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情重創全球經濟，台灣連鎖家具龍頭詩肯表示，目前貨源與庫存都無虞，加上家具產業仍有基本需求在，抗風險的能力更高，2月營收較去年成長，3月至今的營運也都比去年同期更佳，營運已具經濟規模，期盼疫情平息後，可在恐慌心態結束的反轉性爆發中獲益。
不過，對於今年營運，詩肯也轉為保守看待。主要因為今年一月底大陸爆發新冠肺炎，台灣疫情目前也未見和緩趨勢，對於零售消費信心造成的影響難以預估，目前雖主要影響在觀光與餐飲，但未來變化還需要持續觀察。
詩肯主管表示，過去有些消費者對於詩肯品牌的選擇意願不高，最大的原因是由於詩肯門市營造出的氛圍，讓消費者有高價的印象，為因應消費習慣改變，今年將會推動電子商務發展，銷售平台以官網為主，預計上半年啟動計劃。
詩肯指出，推動電子商務後，希望在官網上透過公開的產品價格與消費者溝通，改變消費者印象，並搭配如臉書粉絲團、LINE等社群平台，進一步導客至實體門市，對於營收挹注將更有幫助。
依據財報，詩肯2019年全年營收達18.29億元，年增7.33%，創歷史新高；惟外在大環境不佳、影響台灣消費信心，稅後純益為1.04億元，年減38.09%，每股稅後純益（EPS）為2.25元。董事會並通過配發1.8元現金股利，配發率達八成。
----------------------------------------------------------------------

[10] 國有地上權招標／國泰營區招標時程延後　4月底公告　世貿三館成風向球

▲國泰營區地上權鄰近空軍松山指揮部。（圖／記者湯興漢攝）
記者吳靜君／台北報導
萬眾矚目的信義計畫區世貿三館地上權即將在下周一（30日）開標，然而同樣位在台北市精華區的國泰營區地上權案，原本預期在3月底公告招標，審議小組則延後到4月初才討論，公告招標時間也拖延到4月底，較原先預期晚約一個月，國產署官員表示，世貿三館的標售情形，確實會影響國泰營區的招標內容，世貿三館成風向球。
外界好奇，國泰營區地上權招標案是否因為新冠肺炎（COVID-19）疫情而延後招標。
國產署官員解釋，因為審議小組是每個月召開一次，由於3月的審議小組，國泰營區的議案還未被排入，加

▲燦星旅成立於2003年，為國內第一代網路旅行社大股東燦星網董事會今（27）日決議處分燦星旅行社股權。（圖／資料照）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，燦坤集團旗下「燦星旅行社」今（27）日公告將五成股權出售給台南幫第三代侯佑霖及鄭寶蓮。侯佑霖從創設萬通人力公司，再到亞果遊艇集團，短短五年間，躍居我國遊艇產業的明日之星，這次入主燦星旅危機入市，引發業界高度關注。
燦星旅成立於2003年，為國內第一代網路旅行社，過去受惠於燦坤3C連鎖通路據點優勢，以類似店中店概念設立實體門市，主要經營日本線、東南亞自由行，以及國旅行程等路線。燦坤集團旗下燦星網今（27）日發佈重大訊息公告，因外界經營環境變化，整體營運表現未如預期持續虧損，已與公開收購人簽訂股份買賣協議書，董事會決議處分燦星旅股權。
時間拉回1993年，甫拿下企管碩士學位的侯佑霖，進入父親侯慶輝與吳修齊三弟吳俊傑、法人代表鄭寶蓮創設的萬通人力公司，一腳踏入外勞仲介的業務領域，二十多年來業務逐步擴張，原本只有二名員工的公司，已翻身成為旗下坐擁三十餘家子公司的企業集團。
▲亞果遊艇集團董事長侯佑霖，在2014年與世界前四大遊艇製造商嘉鴻合資創立亞果遊艇集團。(圖／翻攝自亞果遊艇集團官網)
他在2014年與世界前四大遊艇製造商嘉鴻合資，創立亞果遊艇集團，現職為亞果遊艇集團董事長，將興趣作為事業來經營，不僅在高雄展覽館旁的港灣海域，打造「亞果遊艇會」，專門代理國內外的遊艇買賣，無論是全新的、或是二手遊艇，成為業內重要的交易平台。
近五年來，侯佑霖透過自行創設及併購，與合資夥伴、嘉鴻執行長呂佳揚建立起遊艇休憩產業王國，目前旗下已有亞果國際潛水、亞果遊艇開發、亞澎遊艇開發、亞青海洋文創、錚儀遊艇、樂活海洋學院等六大集團企業。
透過亞果遊艇會的營運，侯佑霖以俱樂部形式，提供熱愛海洋活動的金字塔頂端人士全方位的服務　以遊艇休憩產業一條龍的營運模式，成功攻佔富商巨賈的心佔率，進而推升公司營運實績。
▲亞果遊艇集團以遊艇休憩產業一條龍的營運模式，成功攻佔富商巨賈的心佔率。(圖／翻攝自亞果遊艇集團官網)
在遊艇俱樂部的平台上，亞果從銷售會員證，進而跨足遊艇租賃、停 泊、維護、代管、派遺、代理銷售、遊艇駕訓、海域潛水教練等領域，透過旗下六大集團企業的交叉資源運用，開啟多元化的產業經營模式。
亞果遊艇集團2017年11

▲正義國宅都更案「Diamond Tower」1月16日舉行上梁典禮，目前持續興建工程，公開推案時間未定。（圖／馬景平攝）
圖文／CTWANT
春暖花開的3月，正是出門賞屋好時機。然而即將登場的房市329檔期，本該是敲鑼打鼓百案齊發，今年卻出奇地冷清安靜。
據《住展雜誌》預估，金鼠年329檔期六都總推案量約2600億元，比去年同期大減600多億元，衰退約2成7，創下2012年以來同期新低量。其中，北台灣推案量約1300多億元，年減逾3成。
▲今年329檔期，台北市松山火車站周邊有「松蔦青語」（上、左）及「信松廣場」（右）兩案公開，主打中小坪數。（圖／馬景平攝）
疫情蔓延中，去年大鳴大放的台北市豪宅推案，今年329檔期幾乎全面「噤聲」，北台灣建商都以20坪至45坪的中小坪數當主力！
本刊記者日前走訪北台灣多個主要案場，發現在疫情影響下，房產業者多採「拖字訣」，慢慢「雕琢」接待中心、樣品屋，或是不開案，甚至有人辯稱：「現在已經沒有人在推329檔期了。」
▲台北市房價高，329新推案以中小坪數壓低總價，強攻自住市場。（圖／馬景平攝）
「新聯陽實業」副總經理張俊杰表示，「受疫情影響，客戶出門意願降低，現階段所有案件都是小量低調進行，今年329檔期沒有業者大打媒體廣告公開銷售，都是觀望再觀望，在疫情被穩定控制前，維持潛銷狀態。」新聯陽實業是北台灣的不動產代銷巨頭之一。
▲住商不動產企劃研究室經理徐佳馨指出，在疫情期間，開盤怕買氣不佳，因此延後開案未嘗不是另類護盤。（圖／住商不動產提供）
「住商不動產」企劃研究室經理徐佳馨則分析，疫情持續衝擊下，不確定因素很多，「開案非常在意『氣勢』，開案熱銷先贏一半，萬一開盤買氣不佳，不僅士氣大減，也難向業主交代。此外，由於業者怕後市不佳，開案價格若下修，對初期進場購屋的民眾難以交代，因此延後開案未嘗不是另類護盤。」
《住展雜誌》研發長何世昌解釋，現在「329檔期」區間很大，2月到4月推案也可彈性計入。今年受疫情衝擊，329檔以台北市新建案延推情況較為普遍，但若接待中心已蓋好、人員進駐，在成本壓力下不可能不推，如果接待中心還沒蓋好，的確有建商放緩工程腳步延推。
----------------------------------------------------------------------

[34] 撐不住！燦星網董事會決議：出清

▲台酒防疫清潔專用75%酒精。（圖／業者提供）
記者吳靜君／台北報導
近因新冠肺炎（COVID-19）橫行，社會大眾為降低被傳染風險而紛紛減少戶外活動，且自3月19日起凡曾出國的民眾，需於回國後暫緩捐血28天，因而導致血庫難以維持安全存量。台酒特別號召民眾在329青年節當天，一起到228公園捐血，就送300ml的防疫酒精和台酒酒香泡麵。
台灣菸酒公司身為防疫國家隊一員，台酒董事長丁彥哲親自領軍所屬五大酒廠三班調度產能全開，投入防疫酒精生產，另亦觀察疫情發展狀況隨時善盡企業社會責任。
此外，為協助解決血荒窘境，特結合臺北捐血中心，選定329青年節在228和平紀念公園舉辦「熱血台啤 廉手防疫」捐血活動，號召大專院校師生及民眾，凡是捐血250cc即可獲得「廉手防疫福袋」，內裝「台酒防疫清潔用酒精300毫升」一瓶及「台酒酒香泡麵」。
活動現場還邀請台啤籃球隊蔣淯安等球星與民眾互動，並贈送球隊簽名海報，另結合活動主題，透過影音播放等方式讓民眾體會反貪倡廉對企業誠信、永續發展的重要性。
----------------------------------------------------------------------

[46] 台指期失守9600點　分析師：短彈格局、震盪往下機率大

▲台指期3月27日走勢。（圖／取自期交所）
記者余弦妙／台北報導
台指期今（27）日受美股激勵，開盤大漲268點，來到9899點，早盤一度站上9910點，漲279點，不料靠近萬點附近賣壓湧現，隨即一路震盪往下，尾盤甚至翻黑，終場以9583點作收，下跌48點，與現貨相較，逆價差115.92點。
展望後市，統一期貨投顧部分析師劉恆誌分析，近日反彈都是短期效應，還是要回歸疫情是否獲得控制，因此未來走勢將會震盪往下。
劉恆誌表示，昨晚美國公布的失業人口數據非但沒讓美股下跌，反倒是道瓊指數出現大漲，其實這樣的狀態是不太合理，不過市場投資人的預估是，當數據越糟糕，聯準會就可能有機會再度出手救市，因此促成美股大漲。
劉恆誌說，而今天台指期受昨晚美股大漲影響，開盤也出現大幅上漲，不過就在靠近萬點附近就出現賣壓，收盤甚至跌破9600點，這其實是因為目前的反彈都是短期反彈，都是因為經濟刺激政策所致，且這些救市措施並無法讓疫情減緩，因此整體市場都還是短彈格局，未來仍舊是偏空，且外資目前就籌碼部分來看，也並未由

▲宏泰人壽大樓。（圖／宏泰人壽提供）
財經中心／台北報導
新冠肺炎（COVID-19）疫情蔓延，疫情指揮中心日前表示，考量病毒傳播風險，即日起室內超過百人以上、室外超過500人以上的公眾集會活動建議停辦。宏泰人壽配合政府政策，原定於3月26日於台中林酒店舉辦400至450人參與的「2019宏泰人壽業務大會頒獎典禮」也因故延期，公司高層決議靜待新冠肺炎疫情過去後擇日再辦。
宏泰人壽總經理湯維華表示，回顧以往宏泰人壽的年度業務大會頒獎典禮，平均參與人數約400至500人，活動場地多為星級飯店會議廳，參與者包含公司高層、工作人員、壽險業務員及受獎者的親友和媒體。今年礙於新冠肺炎疫情考量大家的安全，不希望因為活動聚集發生病毒感染，以免造成更大的損失及社會問題。待疫情過去將擇日舉辦，頒獎表揚業績卓越的壽險業務員。
此外，社會大眾詢問醫療險的人數增多，反而增加了壽險業務員和客戶之間寒暄聯繫的話題；當有防疫措施及相關報導時，壽險業務員也會跟客戶以通訊軟體分享，促成彼此雖不見面但頻率卻是增加的互動機會。湯維華也呼籲，為了自身健康，近期盡可能減少群聚社交，並注意自身防疫、照顧好自己的身體健康。
----------------------------------------------------------------------

[57] Galaxy S20銷量受疫情衝擊　外媒指稱可能不到同期間S10的一半　

▲Galaxy S20系列的總銷量也受到疫情衝擊。（圖／記者邱倢芯攝）
記者謝仁傑／綜合報導
三星對Galaxy S20到目前為止的表現顯然並不滿意，韓國分析師預估，S20不如去年同期S10的銷量。不過，三星尚未正式發佈銷售數字，預計本季晚些時候將會發佈。
值得注意的是，Galaxy S10的銷售量也不如前代的產品好。
隨著新冠病毒（COVID-19）疫情的肆虐，對全球主要市場經濟造成了重大損害，成為今年三星的不可抗逆因素。市場觀察者降低了對三星今年業績的期望。不過根據先前的報告，售價為1,400美元（約新台幣42,000元）的Galaxy S20 Ultra，約佔S20系列總體銷量的50％。
----------------------------------------------------------------------

[58] 台股收盤下跌37點！失

▲美國通過史上最大規模紓困方案，準備挽救岌岌可危的經濟。（圖／達志影像／美聯社）
全美新冠肺炎（COVID-19）疫情持續升溫，截至台灣時間3月27日早上5點統計，已累計82,404個確診，超越中國大陸，成為世界最多確診數國家，並有1,178例死亡；其中，紐約就超過3萬例確診最為嚴重，有如失速列車，「美版武漢」。
受疫情影響，投資人恐慌情緒升溫，美股在前二週急速墜入熊市，道瓊指數並於3月23日跌至18,591.93點，已接近川普於2016年11月8日普擊敗希拉蕊登上美國總統大位時的18,322.74點（也稱為川普馬其諾防線）。
為此，川普擔憂股市崩跌及經濟衰退影響其選情，先是施壓聯準會（Fed）一口氣降四碼至零利率，並推出史無例的無限量「量化寬鬆政策，QE」救市；除此，川普政府還推出規模達2兆（美元，以下同）的經濟振興計畫（Relief Package），已於當地時間3月25日獲參議院通過，並準備送眾議院審查，通過後再送交美國總統川普簽署生效。
▲美國民眾開始囤貨。（圖／達志影像／美聯社）
兩兆美元紓困案　美國史上最龐大
這次的刺激方案規模號稱美國史上最大，遠超過歐巴馬政府在2008年金融海嘯時的8,000億紓困方案。此項紓困案具體內容包括：
(一)直接發錢給民眾，並分二期發放，第一期大人每人1,200美元、小孩500美元，四口之家可領3,000美元，所需金額約5,000億。
(二)白宮規劃提撥經費給聯邦機構，約458億。
(三)各洲與地方政府的補助款，共1,500億。
(四)針對陷入困境的產業提供流動性補助，約5,000億、針對中小企業貸款約3,500億、醫院補助款約1,300億。
(五)擴大失業保險補助，經費約2,500億。
(六)另140億用於藥品研發及口罩、手套、隔離衣與呼吸器支出。
▲封城的加州街頭行人稀疏。（圖／路透）
封城禁足下經濟半停擺　衰退恐近在呎尺
因大多數美國家庭幾乎沒有積蓄，若失業則沒有錢可繳房租或其他帳單及日常消費。因此，包括諾貝爾經濟獎得主在內的經濟學家認為直接發放現金給人民，是用來減輕1930年代以來最嚴重經濟衰退的最有效措施之一，但也凸顯美國所臨的經濟威脅有多大。
此外，主要城市的服務業大規模裁員，則夏天裁員可能會達到300 萬人；失業率將從2月的3.5%低點瞬間上升至9%。美國財政部長努勤（Steven Mnuchin）更嚴重地指出

▲歐美國家因為輕忽疫情，現在多數陷入大規模感染。（圖／路透）
●錢世傑Dr. J／低調的法律及投資研究者
無限QE
歐美國家的漫不經心，讓疫情持續嚴重發酵，昔日密切的國與國關係不得不暫時關閉了門戶，首當其衝當然是航空業、旅宿業，其他產業當然也不遑多讓，需求減少，就算恢復了產能，也要一段時間才會恢復昔日榮景。
▲台股連漲三天。（圖／tradingeconomics.com）
不斷插入的強心針
很多朋友會問：教授，還會再漲嗎？
連續三日大漲，散戶不請自來；會詢問這個問題的朋友，通常是受不了短期上漲的誘惑，本來還想要等跌了之後再買，但問題就在於不跌，等到過了好幾天漲不停，受不了而買進。結果事與願違，一買進就下跌，似乎全世界的人都偷偷地在背後看著你操作。
美國已經成了武漢疫情的重災區，川普政府採用灑錢政策來穩定金融市場的信心，甚至於在提出6兆刺激方案後，說出一句名言：「國家已經可以看到隧道的亮光。」（nation can see "light at the end of the tunnel"） 
聽到這一句話，嘴角不自主地上揚，帶著點輕蔑的態度數落著川普的說法：「不要怪我澆川普一盆冷水，到過隧道的人都知道，看到隧道尾端的亮光不難，難在於要走過去的時間很漫長，而且一定要有實際行動才會到。」
川普政府的最大問題在於防疫政策猶豫不決、落實有限，民眾防疫知識不足，甚至於有年輕人說如果得了新冠肺炎，還是要去開趴（If I Get Corona, I Get Corona），也難怪最近的疫情居高不下；換言之，不往曙光走過去，就算把曙光搞得再亮，也只是看到罷了！
強心針效應
我稱之為「強心針效應」，倒在地上已經快掛的人，只好學電影「福爾摩斯」的劇情，瀕臨死亡的福爾摩斯，被華生醫生打入一針可能是腎上腺素的藥劑，馬上活蹦亂跳地在車廂中跑來跑去。但骨折還是骨折、傷痛還是要慢慢療傷，該面對的還是要面對，譬如說四月十日前公布三月份營收，醜媳婦還是要面對公婆。
▲新冠病毒確診數與增加率。（資料來源／WHO）
不管如何，時間無法回到過去，美國作為重要經濟大國，疫情已經嚴重不可收拾，我們期待這個大國至少也要有點兒作為，而不是插支經濟強心針就結束；否則，疫情沒有反轉，許多國家將禁止美國人入境，對於全世界經濟的傷害將會持續下去，至於美國老年人是否願意犧牲，這並不是其他國家防疫政策考量的重點。
結論：股災看

▲道瓊從24日回漲後持續3天，幅度高達21％。（圖／路透社）
記者張靖榕／綜合外電報導
美股周四（26日）開盤延續前2天漲勢，道瓊終場漲幅衝破6％，創下90年以來3日最高漲幅，從周一低點後，至今已回漲21％，迅速回歸牛市。
道瓊指數終場上漲1351.62點，或6.38％，收在22552.17點；那斯達克指數上漲413.24點，或5.6％，收在7797.54點；標普500指數終場則上漲154.51點，或6.24％，以2630.07點作收。
美股因新冠肺炎衝擊而結束08年金融危機之後的牛市，正式進入熊市；然而國會民主和共和兩黨在爭執後重新談定，通過2兆美元的天價經濟刺激方案，為市場注入一劑強心針。連3天共「回血」21％的漲勢將美股重新帶上2萬點大關，是自1931年以來的最強3日漲幅，投資人信心增加，也連帶拉動歐股上揚。
與此同時，疫情衝擊使得企業工廠停擺，美國已經高達328萬人失業，川普政府將會運用大撒幣方式先暫時失業者滿足失業者在非常時期的經濟來源，同時避免經濟垮台。另外，美國經濟命脈之一的波音公司也大漲14％，過去4個交易日更大漲90％，得利於前一波的紓困方案，政府對航空業提供了580億美元。
歐股方面，英國FT100上漲127.53點，或2.24％，收在5815.73點；NQ德國指數上漲27.62點，或3.25％，收在876.99點；法國CAC40指數上漲111.28點，或2.51％，收在4543.58點。
----------------------------------------------------------------------

[89] 行政院加碼補助觀光業　雄獅、六福董座齊按讚：「擋住失業潮！」

▲行政院今（26）日宣佈加碼觀光旅宿業薪資補助方案，雄獅旅遊董事長王文傑表示，感謝政府重視產業聲音。（圖／雄獅旅遊提供）
記者林淑慧／台北報導
行政院今（26）日宣佈加碼觀光旅宿業薪資補助方案，雄獅旅遊董事長王文傑、六福旅遊董事長莊豐如均表示，感謝政府重視產業聲音，快速指示調整產業紓困政策，挽救疫情衝擊下的海嘯第一排，薪資補助倍增可望擋住失業潮。
新冠肺炎疫情升溫，台灣加強入出境管制，重擊觀光旅遊產業鏈，旅行社、飯店民宿業全面受影響，行政院今天拍板對觀光旅宿業的艱困企業薪資補助方案，決定加碼一倍，若企業維持員工8成薪，將補助員工每月薪資4成、上限